# CLM SeasonalDecidOnset Phenology Onset/Offset in Python

This notebook re-implements the CLM5 SeasonalDecidOnset subroutine using Python. It demonstrates how empirical phenology onset and offset can be calculated from daily temperature and photoperiod.


## Algorithm Overview
- **Inputs:** daily minimum temperature, maximum temperature, photoperiod, and day-of-year arrays.
- **Parameters:**
  - `photoperiod_threshold` (hours)
  - `temperature_threshold` (°C)
  - `accumulation_days` (running window length)
  - `onset_flag` (`True` for onset, `False` for offset)
- For each day, compute binary forcing flags for photoperiod and temperature above thresholds.
- Maintain running sums over the last `accumulation_days`.
- **Onset:** first day when both sums are ≥ `accumulation_days`.
- **Offset:** first day (after onset) when both sums are < `accumulation_days`.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

params = {
    'photoperiod_threshold': 12.0,  # hours
    'temperature_threshold': 5.0,   # °C
    'accumulation_days': 7,         # days
}


In [ ]:
def seasonal_decid_onset(tmin, tmax, photoperiod, doy, *,
                           photoperiod_threshold=12.0,
                           temperature_threshold=5.0,
                           accumulation_days=7,
                           onset_flag=True):
    """Empirical onset/offset following CLM SeasonalDecidOnset.

    Returns the day-of-year index for the event or -1 if not found.
    """
    tmin = np.asarray(tmin)
    tmax = np.asarray(tmax)
    photoperiod = np.asarray(photoperiod)
    doy = np.asarray(doy)
    if not (len(tmin) == len(tmax) == len(photoperiod) == len(doy)):
        raise ValueError('All inputs must have the same length')

    tmean = 0.5 * (tmin + tmax)
    phot_flag = (photoperiod >= photoperiod_threshold).astype(int)
    temp_flag = (tmean >= temperature_threshold).astype(int)

    n = len(doy)
    phot_sum = np.zeros(n, dtype=int)
    temp_sum = np.zeros(n, dtype=int)

    for i in range(n):
        start = max(0, i - accumulation_days + 1)
        phot_sum[i] = phot_flag[start:i+1].sum()
        temp_sum[i] = temp_flag[start:i+1].sum()

    if onset_flag:
        for i in range(n):
            if phot_sum[i] >= accumulation_days and temp_sum[i] >= accumulation_days:
                return int(doy[i])
    else:
        triggered = False
        for i in range(n):
            if not triggered and phot_sum[i] >= accumulation_days and temp_sum[i] >= accumulation_days:
                triggered = True
            elif triggered and phot_sum[i] < accumulation_days and temp_sum[i] < accumulation_days:
                return int(doy[i])
    return -1


In [ ]:
# Example synthetic data
n_days = 60
doy = np.arange(1, n_days + 1)
tmin = np.linspace(-5, 15, n_days)
tmax = tmin + 10
photoperiod = 10 + 4 * np.sin(2 * np.pi * (doy - 80) / 365)

onset_day = seasonal_decid_onset(tmin, tmax, photoperiod, doy, **params, onset_flag=True)
offset_day = seasonal_decid_onset(tmin, tmax, photoperiod, doy, **params, onset_flag=False)
print('Onset day:', onset_day)
print('Offset day:', offset_day)


In [ ]:
plt.figure(figsize=(8,3))
plt.plot(doy, (tmin+tmax)/2, label='Tmean')
plt.plot(doy, photoperiod, label='Photoperiod')
if onset_day != -1:
    plt.axvline(onset_day, color='green', linestyle='--', label='Onset')
if offset_day != -1:
    plt.axvline(offset_day, color='red', linestyle='--', label='Offset')
plt.xlabel('Day of Year')
plt.legend()
plt.tight_layout()
plt.show()


## Conclusion
The function `seasonal_decid_onset` reproduces the CLM SeasonalDecidOnset logic in Python, allowing onset and offset dates to be computed from time series of daily weather variables.

## Testing the Module

In [ ]:
# Test 1: below thresholds
tmin = np.full(10, -5)
tmax = np.full(10, 0)
photoperiod = np.full(10, 8)
doy_test = np.arange(1, 11)
assert seasonal_decid_onset(tmin, tmax, photoperiod, doy_test, **params, onset_flag=True) == -1
assert seasonal_decid_onset(tmin, tmax, photoperiod, doy_test, **params, onset_flag=False) == -1
print('Test 1 passed')

# Test 2: step function trigger on day 6
tmin = np.concatenate([np.zeros(5), np.full(5, 6)])
tmax = tmin + 5
photoperiod = np.concatenate([np.full(5, 11), np.full(5, 13)])
doy_test = np.arange(1, 11)
params_step = {'photoperiod_threshold': 12.0, 'temperature_threshold': 5.0, 'accumulation_days': 1}
assert seasonal_decid_onset(tmin, tmax, photoperiod, doy_test, **params_step, onset_flag=True) == 6
print('Test 2 passed')

# Test 3: sliding window with accumulation_days=3
tmin = [0,6,6,6,0,0]
tmax = [10,12,12,12,10,10]
photoperiod = [11,12,12,12,11,11]
doy_test = np.arange(1,7)
params_sw = {'photoperiod_threshold': 12.0, 'temperature_threshold': 5.0, 'accumulation_days': 3}
assert seasonal_decid_onset(tmin, tmax, photoperiod, doy_test, **params_sw, onset_flag=True) == 4
print('Test 3 passed')
